In [0]:
import pandas as pd
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
!ls '/content/gdrive/My Drive/Data challenge/code'

ls: cannot access '/content/gdrive/My Drive/Data challenge/code': No such file or directory


In [0]:
# import single json file as dataframe
df = pd.read_json('/content/gdrive/My Drive/Data challenge/code/airlines-1558527599826.json', orient = 'records', lines = True)

In [0]:
# disply available columns
df.columns

Index(['created_at', 'id', 'id_str', 'text', 'display_text_range', 'source',
       'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'quote_count', 'reply_count',
       'retweet_count', 'favorite_count', 'entities', 'extended_entities',
       'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang',
       'timestamp_ms', 'retweeted_status', 'extended_tweet',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status',
       'quoted_status_permalink', 'delete'],
      dtype='object')

In [0]:
# preview of df
df[['id','in_reply_to_screen_name','in_reply_to_status_id','in_reply_to_status_id_str','in_reply_to_user_id','in_reply_to_user_id_str']].head(10)

,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str
0,1.131173e+18,None,NaN,NaN,NaN,NaN
1,1.131173e+18,None,NaN,NaN,NaN,NaN
2,1.131173e+18,AlexanderViol,1.131033e+18,1.131033e+18,3.943766e+08,3.943766e+08
3,1.131173e+18,None,NaN,NaN,NaN,NaN
4,1.131173e+18,None,NaN,NaN,NaN,NaN
5,1.131173e+18,None,NaN,NaN,NaN,NaN
6,1.131173e+18,None,NaN,NaN,NaN,NaN
7,1.131173e+18,easyJet,1.131101e+18,1.131101e+18,3.867690e+07,3.867690e+07
8,1.131173e+18,None,NaN,NaN,NaN,NaN
9,1.131173e+18,BilalEksiTHY,NaN,NaN,7.888827e+17,7.888827e+17


In [0]:
# add with nan filled columns for coversations id's
df['conversation']=df['in_reply_to_status_id'].copy()
# fill nan values with own id to give every tweet a unique conversation id
df.loc[np.isnan(df['conversation']),'conversation']=df[np.isnan(df['conversation'])]['id']

In [0]:
# count of distinct conversations (including single tweets)
len(df['conversation'].value_counts())

12213

In [0]:
for id,conv in df.groupby(['conversation']):
    # print(id)
    display(conv)
    break

,created_at,id,id_str,text,display_text_range,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,extended_entities,favorited,retweeted,possibly_sensitive,filter_level,lang,timestamp_ms,retweeted_status,extended_tweet,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,delete,conversation
512,2019-05-22 13:00:38+00:00,1.131183e+18,1.131183e+18,@British_Airways Website not recognising my bo...,"[17, 58]","<a href=""https://mobile.twitter.com"" rel=""nofo...",0.0,4.585687e+17,4.585687e+17,18332190.0,18332190.0,British_Airways,"{'id': 107259411, 'id_str': '107259411', 'name...",None,None,None,NaN,0.0,0.0,0.0,0.0,0.0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,0.0,NaN,low,en,2019-05-22 13:00:38.268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.585687e+17
